# Las dos Españas

En este notebook vamos a responder a la cuestión de la división política en los diferentes municipios. Para ello, vamos a cargar las votaciones por municipio o sección, y vamos a aplicar una técnica de reducción de dimensiones para ver qué distribución poblacional obtenemos. Para poder visualizar mejor los datos, en algunos momentos haremos la división también excluyendo grandes ciudades, o analizándolas por separado.

In [ ]:
pip install ..

In [ ]:
import os
import pandas as pd
import numpy as np
import bokeh as bk
import holoviews as hv

import matplotlib.pyplot as plt
import matplotlib as mpl

import scanpy as sc
import eec

from bokeh.models import HoverTool, LinearColorMapper, LogColorMapper, Label
hv.extension('bokeh')
from bokeh.io import show, output_file, push_notebook, curdoc
from bokeh.models import ColumnDataSource, Select, HoverTool, Panel, Tabs
from bokeh.plotting import figure
from bokeh.models.widgets import MultiSelect, CheckboxGroup, Dropdown
from bokeh.layouts import column , row
from bokeh.themes import Theme

from tqdm import tqdm, tqdm_notebook
import geopandas as gpd

import umap
import scipy.stats as sts
from scipy.stats import linregress as lr

import panel as pn
import panel
import param

import warnings
warnings.filterwarnings('ignore')

current_dir = os.path.dirname(os.getcwd())
current_dir

In [ ]:
año, mes = '2019', '11'

In [ ]:
municipios_gpd = gpd.read_file(current_dir + 
            f'/datos/mapas_elecciones/porcentaje_mas_votado_{año}_{mes}_municipios.json', encoding='utf-8')

secciones_gpd = gpd.read_file(current_dir + 
            f'/datos/mapas_elecciones/porcentaje_mas_votado_{año}_{mes}_secciones.json', encoding='utf-8')

provincias_gpd = gpd.read_file(current_dir + 
           f'/datos/mapas_elecciones/porcentaje_mas_votado_{año}_{mes}_provincias.json', encoding='utf-8')

df_partidos = pd.read_pickle(current_dir + f'/datos/congreso_{año}_{mes}/resultados_candidaturas.pickle')

Asignamos las provincias autónomas a cada fila

In [ ]:
prov_municipios = [i[:2] for i in municipios_gpd['CUMUN'].values]
prov_secciones = [i[:2] for i in secciones_gpd['CUSEC'].values]

dict_prov = dict(zip(provincias_gpd['CPRO'].values, provincias_gpd['Nombre'].values))

municipios_gpd['CPRO'] = [dict_prov[i] for i in prov_municipios]
secciones_gpd['CPRO'] = [dict_prov[i] for i in prov_secciones]

In [ ]:
dict_prov = {'Araba/Álava': 'Euskadi', 'Albacete': "Castilla la Mancha", 'Alicante': "Comunidad Valenciana",'Almería': 'Andalucía',
 'Ávila': "Castilla y León", 'Badajoz': "Extremadura", 'Balears, Illes': "Baleares", 'Barcelona': "Cataluña",
 'Burgos': "Castilla y León", 'Cáceres': "Extremadura", 'Cádiz': 'Andalucía', 'Castellón/Castelló': "Comunidad Valenciana",
 'Ciudad Real': "Castilla la Mancha", 'Córdoba': 'Andalucía', 'Coruña, A': "Galicia",
 'Cuenca': "Castilla la Mancha", 'Girona': "Cataluña", 'Granada': 'Andalucía',
 'Guadalajara': "Castilla la Mancha", 'Gipuzkoa': 'Euskadi', 'Huelva': 'Andalucía', 'Huesca': 'Aragón',
 'Jaén': 'Andalucía', 'León': "Castilla y León", 'Lleida': "Cataluña",
 'Rioja, La': "La Rioja", 'Lugo': "Galicia", 'Madrid': "Madrid", 'Málaga': 'Andalucía',
 'Murcia': "Murcia", 'Navarra': "Navarra", 'Ourense': "Galicia", 'Asturias': "Asturias",
 'Palencia': "Castilla y León", 'Palmas, Las':'Canarias', 'Pontevedra': "Galicia", 'Salamanca': "Castilla y León",
 'Santa Cruz de Tenerife': 'Canarias', 'Cantabria': "Cantabria", 'Segovia': "Castilla y León", 'Sevilla': "Andalucía",
 'Soria': "Castilla y León", 'Tarragona': "Cataluña", 'Teruel': 'Aragón', 'Toledo': "Castilla la Mancha", 
 'Valencia/Valéncia': "Comunidad Valenciana", 'Valladolid': "Castilla y León", 'Bizkaia': 'Euskadi',
 'Zamora': "Castilla y León", 'Zaragoza': 'Aragón', 'Ceuta': 'Ceuta', 'Melilla': 'Melilla'}

dict_num_prov = {list(dict_prov.keys())[i]:i for i in range(len(dict_prov))}

In [ ]:
municipios_gpd['Autonomia'] = [dict_prov[i] for i in municipios_gpd['CPRO']]
secciones_gpd['Autonomia'] = [dict_prov[i] for i in secciones_gpd['CPRO']]

In [ ]:
df_partidos = df_partidos[[i for i in df_partidos.columns if i not in ['codigo provincia', 'codigo municipio', 'numero distrito',
       'codigo seccion', 'censo escrutinio', 'votos candidaturas',
       'votos blancos', 'votos nulos', 'codigo', 'PP-FORO']]]
df_partidos = df_partidos.fillna(0)
columnas_partidos = [i for i in df_partidos.columns if 'porcentaje_' + i in municipios_gpd.columns]

In [ ]:
df_propiedades = pd.read_csv(current_dir + '/datos/info_partidos/info_partidos.txt', sep='\t', quoting=3
                            ).set_index('Siglas')
dict_colores = dict(zip(df_propiedades.index.values, df_propiedades['Color'].values))

Primero vamos a sacar el porcentaje de votos para poder analizar bien las relaciones que decía antes.

In [ ]:
for partido in columnas_partidos:
    secciones_gpd['porcentaje_{}'.format(partido)] = 100 * secciones_gpd[partido] / secciones_gpd['votos candidaturas']
    municipios_gpd['porcentaje_{}'.format(partido)] = 100 * municipios_gpd[partido] / municipios_gpd['votos candidaturas']

## Análisis de UMAP y clustering
En este análisis vamos a crear plots en los que reduciremos la información electoral (porcentajes de votos por partido, porcentaje de abstenciones, etc.) a dos dimensiones. Para ello vamos a desarrollar varias funciones.
* `calcular_UMAP`: Obtiene la matriz de datos (en un array), y la introduce en el embedder de UMAP. Con esto obtenemos una matriz con dos columnas, que será la que dibujemos en el gráfico.
* `calcular_leiden`: Obtiene, a partir de la matriz original de datos, una matriz que asigna a cada elemento una etiqueta, de modo que elementos similares tienen la misma etiqueta. Buscamos obtener unas 10-20 etiquetas para todos los datos, y así poder ver tendencias dentro del conjunto de datos.
* `dibujar_UMAP`: Genera un plot con toda la información disponible del análisis. Tiene una pestaña con opciones para poder seleccionar, que permite visualizar información sobre los votos por cada partido, los partidos más votados, o los grupos de elementos similares (`leiden`).

Tods estas funciones están externalizadas, y condensadas en una sola: `dibujar_UMAP_votos_autonomia`.

## A analizar!
Ahora que todo está preparado, vamos a empezar a sacar las figuras para hacer el análisis. Corre cada celda para hacer el análisis. Es probable que si corres una segunda celda, la primera deje de cambiar las opciones del menú. Corre la celda de nuevo si te pasa eso.

Vamos a coger cada uno de los datos y los vamos a ir analizando individualmente.
`seed` es la semilla para replicar los resultados.

### Análisis de secciones
Este análisis muestra 8 grupos principales. El más pequeño de todos [grupo 27] pertenece a melilla. De izquierda a derecha (puede que no), observamos los grupos [24,25] que pertenecen a Cantabria, [12,13,14,15] Canarias, [4,10,11] Comunidad Valenciana, [16,17,18,19] Cataluña, [20, 21, 22, 23] Navarra, [0,1,2] Euskadi, y el resto, en el centro, que pertenecen al resto de todas las comunidades. Este cluster central puede albergar secciones de las comunidades mencionadas anteriormente, pero es extraño que sea así. Todas las comunidades han sido separadas independientemente por la presencia de partidos regionales, como ERC, JuntsxCAT, EH Bildu, NA+, CCa, u otros.

Dentro de cada grupo grande, sin embargo, podemos ver tendencias claras, que iremos desglosando.

#### Euskadi
Euskadi se divide en 3 grupos principales:
* En el grupo 2, los dos primeros partidos son PNV y Bildu, y el tercer partido es de izquierdas. Si tiramos hacia la punta de la izquierda, hay partidos con más votos de Bildu, y hacia la derecha más votos de PNV.
* El grupo 1 consisste principalmente en secciones con menos presencia de Bildu. Hacia la izquierda la segunda fuerza es PP, y hacia la izquierda es PSOE. Las secciones suelen ser pueblos más grandes, o incluso ciudades.
* En el grupo 0 hay una mayoria del PSOE. La isleta de más a la izquierda son secciones con mayoria de PP y VOX.
En este caso se recomienda visualizar el partido más votado, porque revela bastante bien la información.

#### Navarra
Se diferencian 4 grupos principales.
* El grupo 20 tiene fuerza progresista como PSOE o UP, o a NA+. Más a la derecha indica mayor fuerza de PSOE / UP, y más a la izquierda indica mayor fuerza de NA+.
* El grupo 21 tiene como primera fuerza a EH Bildu.
* El grupo 22 contiene secciones que han votado a VOX, auqnue no esté entre las 3 fuerzas principales.
* El grupo 23 tiene como fuerza mayoritaria a NA+.

#### Cataluña
En Cataluña hay 4 grupos.
* 19 tiene a Cs como primera fuerza, y tiene la mayor presencia de VOX.
Ahora, de izquierda a derecha, tenemos un gradiente de independentismos. El grupo 18 está constituido principalmente por ERC como primera fuerza y JxCAT como segunda; el grupo 19 está constituido por ERC como primera fuerza y PSOE como segunda; y el grupo 20 está constituido por PSOE o UP como primera fuerza.

#### Canarias
En Canarias hay 4 grupos principales, que los podemos agrupar en presencia de CCa en las tres primeras fuerzas (izquierda, grupos 13 y 14), o ausencia. Los tres grupillos que se forman arriba son secciones con mucha abstención. Entre los grupos 12 y 15 no hay grandes diferencias, aunque se ve que en 12 hay una mayor fuerza de la derecha en la parte sudoeste del cluster.

#### Cantabria
La principal división es PSOE como fuerza mayoritaria (24) o PP (25). En el grupo 25, entre PP y PSOE, están las secciones donde es PRC primera fuerza. Está en 25 porque la segunda fuerza es PP, o VOX es tercera.

#### Melilla
En Melilla se muestran solo los partidos que han votado a CpM. El resto se encuentran en el grupo enorme. En algunas ocasiones hay una distinción en dos grupos: Uno con CpM entre las tres primeras fuerzas, y otro con VOX.

#### Comunidad Valenciana
La división de la Comunidad Valenciana se hace en dos grandes grupos: el grupo 10 contiene una mayoria de derechas, cuanto más a la izquierda más representación de derechas, (culminando en la punta con el trifachito); mientras que el grupo 11 contiene una mayoría de izquierdas. En la cabeza de arriba a la derecha del grupo 10 es compromís primera fuerza.

#### Resto del grupos
El resto de grupos representan al resto de autonomías, o a secciones dentro de las autonomías anteriores que no han tenido presencia de partidos regionalistas.
La descripción de grupos principal es la siguiente:
* 9: PP + VOX + Cs
* 10: Como 9, o PP + Cs
* 7: PP como primera fuerza, y con diferencia
* 8: PP como primera fuerza, pero menos diferencia. La segunda o tercera fuerza es Cs o VOX.
* 3: Dos fuerzas del trifachito o PSOE + UP. Las diferencias entre las tres fuerzas mayoritarias es pequeña. Se podrían definir como secciones centristas.
* 5: Parecido a 3.
* 4: PSOE como primera. Cuanto más al centro del grupo grande, Cs es la segunda fuerza, y cuanto más al extremo, UP es la segunda fuerza. 

In [ ]:
df_datos_secciones = eec.crear_df_datos(secciones_gpd, 0.1, 0.2, seed=10, r_min=2, r_max=15, )

In [ ]:
eec.dibujar_UMAP_votos_autonomia(df_datos_secciones, dict_colores, alpha_max=0.95, alpha_min = 0.005, titulo='')

In [ ]:
import param
import panel
import holoviews as hv
import pandas as pd
import numpy as np
import panel
import param
from bokeh.io import curdoc
from bokeh.themes import Theme
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource

import matplotlib.pyplot as plt
import matplotlib as mpl

import umap
import scipy.stats as sts
import scanpy as sc

dict_prov = {'Araba/Álava': 'Euskadi', 'Albacete': "Castilla la Mancha", 'Alicante': "Comunidad Valenciana",'Almería': 'Andalucía',
 'Ávila': "Castilla y León", 'Badajoz': "Extremadura", 'Balears, Illes': "Baleares", 'Barcelona': "Cataluña",
 'Burgos': "Castilla y León", 'Cáceres': "Extremadura", 'Cádiz': 'Andalucía', 'Castellón/Castelló': "Comunidad Valenciana",
 'Ciudad Real': "Castilla la Mancha", 'Córdoba': 'Andalucía', 'Coruña, A': "Galicia",
 'Cuenca': "Castilla la Mancha", 'Girona': "Cataluña", 'Granada': 'Andalucía',
 'Guadalajara': "Castilla la Mancha", 'Gipuzkoa': 'Euskadi', 'Huelva': 'Andalucía', 'Huesca': 'Aragón',
 'Jaén': 'Andalucía', 'León': "Castilla y León", 'Lleida': "Cataluña",
 'Rioja, La': "La Rioja", 'Lugo': "Galicia", 'Madrid': "Madrid", 'Málaga': 'Andalucía',
 'Murcia': "Murcia", 'Navarra': "Navarra", 'Ourense': "Galicia", 'Asturias': "Asturias",
 'Palencia': "Castilla y León", 'Palmas, Las':'Canarias', 'Pontevedra': "Galicia", 'Salamanca': "Castilla y León",
 'Santa Cruz de Tenerife': 'Canarias', 'Cantabria': "Cantabria", 'Segovia': "Castilla y León", 'Sevilla': "Andalucía",
 'Soria': "Castilla y León", 'Tarragona': "Cataluña", 'Teruel': 'Aragón', 'Toledo': "Castilla la Mancha",
 'Valencia/Valéncia': "Comunidad Valenciana", 'Valladolid': "Castilla y León", 'Bizkaia': 'Euskadi',
 'Zamora': "Castilla y León", 'Zaragoza': 'Aragón', 'Ceuta': 'Ceuta', 'Melilla': 'Melilla'}

dict_num_prov = {list(dict_prov.keys())[i]:i for i in range(len(dict_prov))}


def calcular_leiden(array, res, subres, seed):
    print('Calculando leiden')

    adata = sc.AnnData(X=np.nan_to_num(array))
    sc.pp.neighbors(adata)
    sc.tl.leiden(adata, resolution=res, random_state=seed)

    if subres > 0:
        array_return = np.zeros(len(adata))
        clusters = list(dict.fromkeys(adata.obs['leiden'].values))
        n_clusters = 0

        for cluster in clusters:
            index_cluster = np.argwhere((adata.obs['leiden'] == cluster).values).flatten()
            subadata = adata[adata.obs['leiden'] == cluster].copy()
            subadata.X = np.nan_to_num(subadata.X)
            sc.pp.neighbors(subadata)
            sc.tl.leiden(subadata, resolution=subres, random_state=seed)
            array_return[index_cluster] = subadata.obs['leiden'].values.astype(int) + n_clusters
            n_clusters += len(list(dict.fromkeys(subadata.obs['leiden'])))

        return array_return
    else:
        return adata.obs['leiden'].values.astype(int)


def map_sizes(r, min_pop, max_pop, r_min=2, r_max=13, ):
    return r_min + (r - min_pop) / (max_pop - min_pop) * (r_max - r_min)


def calcular_UMAP(df, seed):
    print('Calculando UMAP')
    array_porcentajes = df.loc[:,
                        [i for i in df.columns if (i.startswith('porcentaje_')) & ((not i[-1].isdigit()))]].values
    array_porcentajes = np.nan_to_num(array_porcentajes)

    reducer = umap.UMAP(transform_seed=seed)
    embedding = reducer.fit_transform(array_porcentajes)

    return embedding


def crear_df_datos(df, resolucion, subresolucion, seed, r_min=2, r_max=13, ):
    embedding = calcular_UMAP(df, seed)
    array = df[[i for i in df.columns
                if (i.startswith('porcentaje_')) and (i not in [f'porcentaje_{x}' for x in range(1, 6)])]].copy().values
    
    df['leiden'] = calcular_leiden(array, resolucion, subresolucion, seed)
    min_pop, max_pop = np.power(min(df['censo escrutinio'].values), 0.05), np.power(max(df['censo escrutinio'].values),
                                                                                    0.05)
    df_datos = pd.DataFrame(dict({'x': embedding[:, 0], 'y': embedding[:, 1],
                       'partido_1': df['partido_1'].values,
                       'partido_2': df['partido_2'].values,
                       'partido_3': df['partido_3'].values,
                       'autonomia': df['Autonomia'].values, 'provincia': df['CPRO'].values,
                       'color': np.array(['#000000'] * len(df)), 'Nombre': df['Nombre'].values,
                       'poblacion': df['censo escrutinio'].values,
                       'tamano': np.array([map_sizes(np.power(i, 0.05), min_pop, max_pop, r_min, r_max)
                                           for i in df['censo escrutinio'].values]),
                       'alpha': np.array([0.99 for _ in df['censo escrutinio'].values]),
                       'alpha_line': np.array([0.45 for _ in df['censo escrutinio'].values]),
                       'leiden': df['leiden'].values},
                      **{i: df[i].values for i in df.columns if (i.startswith('porcentaje_'))}))

    return df_datos

In [ ]:
def dibujar_UMAP_votos_autonomia(df, dict_colores, alpha_max=0.95, alpha_min=0.00, titulo=''):
    """
    Estrategia para hacer el update. Anteriormente, la función para actualizar la figura generaba una figura nueva.
    Esto era problemático porque cada vez que se hacía una nueva selección se reestablecía la figura, dejándola en
    la posición y zoom de inicio. Para que sólo se cambien los datos, hay que emplear el método
    source.stream(diccionario, número_de_puntos)
    En este caso, source tiene que ser un ColumnDataSource. No funciona con un DataFrame.

    Para hacer entonces la figura y updatear, la estrategia a seguir es la siguiente.
    1) Crear la figura con una función interna crear_fig (NO METODO). Esta función crea el Datasource (NO SE podría importar
    como atributo de la función inicial). La función retorna la figura y el DataSource, que ahora son atributos de clase.
    2) Para updatear la figura hay que crear un diccionario con la misma forma que el DataSource (NO SE PUEDEN CREAR
    NUEVAS COLUMNAS). Este diccionario, con los elementos cambiados, se emplea en source.stream(dict, N).
    N es el número de puntos que hay en la figura. Si se ponen menos puntos, se eliminan de la figura, y si se ponen
    más se generan por encima de los que ya hay.
    3) Como otro método de importación, se puede trabajar siempre con un DataFrame. Es decir, tanto la creación del
    DataSource (ColumnDataSource(df)) como la actualización del source (source.stream(sub_df, len(sub_df)) pueden
    emplearse DataFrames, siempre que se mantengan las mismas columnas y puntos.

    """

    class UMAPPlotClass(param.Parameterized):
        lista_opciones = ['leiden'] + ['partido_1', 'partido_2', 'partido_3'] + ['provincia', 'autonomia'] + \
                         [i for i in df.keys() if (i.startswith('porcentaje_')) &
                          (i not in ['porcentaje_1', 'porcentaje_2', 'porcentaje_3'])]
        lista_autonomias = sorted(list(dict.fromkeys(df['autonomia'])))

        obj_opciones = param.ObjectSelector(default='partido_1', objects=lista_opciones)
        obj_autonomias = param.ListSelector(default=lista_autonomias, objects=lista_autonomias)

        def color_mapper(self, c2, c1, mix):
            if np.isnan(mix): mix = 0
            c1 = np.array(mpl.colors.to_rgb(c1))
            c2 = np.array(mpl.colors.to_rgb(c2))
            return mpl.colors.to_hex((1 - mix) * c1 + mix * c2)

        def crear_fig():  # Esto técnicamente está mal pero funciona, así que ni lo toco
            fig = figure(plot_height=700, plot_width=700, tools='box_zoom,reset,pan,wheel_zoom,lasso_select,undo,redo',
                         sizing_mode='scale_width', output_backend="webgl", toolbar_location='right')

            hover_UMAP = HoverTool(
                tooltips="""
                    <div><span style="font-size: 17px; font-weight: bold;">@Nombre</span></div>
                    <div><span style="font-size: 12px;">@provincia (@autonomia), @poblacion</span></div>
                    <div><span style="font-size: 14px; font-weight: bold;">@partido_1</span>
                    <span style="font-size: 13px;">@porcentaje_1 %</span></div>
                    <div><span style="font-size: 14px; font-weight: bold;">@partido_2</span>
                    <span style="font-size: 13px;">@porcentaje_2 %</span></div>
                    <div><span style="font-size: 14px; font-weight: bold;">@partido_3</span>
                    <span style="font-size: 13px;">@porcentaje_3 %</span></div>
                    <div><span style="font-size: 14px; font-weight: bold;">% Abstención</span>
                    <span style="font-size: 13px;">@porcentaje_abstencion %</span></div>
                    <div><span style="font-size: 14px; font-weight: bold;">Grupo</span>
                    <span style="font-size: 13px;">@leiden</span></div>""",
            )

            fig.add_tools(hover_UMAP)
            fig.axis.visible, fig.xgrid.visible, fig.ygrid.visible = False, False, False
            source = ColumnDataSource(df)
            fig.scatter('x', 'y', source=source, line_alpha='alpha_line', line_width=0.3, line_color="#000000",
                        size='tamano', color='color', alpha='alpha')

            return fig, source

        UMAP, source = crear_fig()

        @panel.depends('obj_opciones', 'obj_autonomias')
        def update_df_datos(self):
            dict_color_leiden = {0: '#5F4690', 1: '#1D6996', 2: '#38A6A5', 3: '#0F8554', 4: '#73AF48',
                                 5: '#EDAD08', 6: '#E17C05', 7: '#CC503E', 8: '#94346E', 9: '#6F4070',
                                 10: '#994E95', 11: '#666666', 12: '#11A579', 13: '#E73F74', 14: '#CF1C90',
                                 15: '#3969AC', 16: '#4b4b8f', 17: '#66C5CC', 18: '#F89C74', 19: '#DCB0F2',
                                 20: '#FE88B1', 21: '#661100'}
            dict_num_autonomias = {'Navarra': 0, 'Canarias': 1, 'Baleares': 2, 'Comunidad Valenciana': 3,
                                   'Asturias': 4, 'Andalucía': 5, 'Aragón': 6, 'Murcia': 7, 'Extremadura': 8,
                                   'Ceuta': 9, 'Cataluña': 10,
                                   'Cantabria': 11, 'Madrid': 12, 'Castilla y León': 13, 'La Rioja': 14, 'Euskadi': 15,
                                   'Melilla': 16,
                                   'Castilla la Mancha': 17, 'Galicia': 18}
            sub_df = df.copy()
            # Primero aplicamos el color
            attr = self.obj_opciones
            if attr in ['partido_1', 'partido_2', 'partido_3']:
                sub_df['color'] = [dict_colores[i] for i in sub_df[attr]]
            elif attr in ['porcentaje_blancos', 'porcentaje_nulos', 'porcentaje_abstencion']:
                max_attr = max(sub_df[attr])
                sub_df['color'] = [self.color_mapper('#000000', "#bbbbbb", i / max_attr) for i in sub_df[attr]]
            elif attr == 'leiden':
                sub_df['color'] = [dict_color_leiden[i % len(dict_color_leiden)] for i in sub_df[attr]]
            elif attr == 'provincia':
                sub_df['color'] = [dict_color_leiden[dict_num_prov[i] % 15] for i in sub_df['provincia']]
            elif attr == 'autonomia':
                sub_df['color'] = [dict_color_leiden[dict_num_autonomias[i] % 19] for i in sub_df[attr]]
            else:
                max_attr = max(sub_df[attr])
                sub_df['color'] = np.array([self.color_mapper(dict_colores[attr.replace('porcentaje_', '')],
                                                              "#f0f0f0", i / max_attr) for i in sub_df[attr]])
            # Ahora aplicamos el alpha de las autonomías
            aut_select = self.obj_autonomias

            sub_df['alpha'] = [alpha_min] * len(sub_df['alpha'])
            sub_df['alpha_line'] = [alpha_min] * len(sub_df['alpha_line'])
            sub_df.loc[sub_df['autonomia'].isin(aut_select), 'alpha'] = alpha_max
            sub_df.loc[sub_df['autonomia'].isin(aut_select), 'alpha_line'] = 0.45

            self.source.stream(sub_df, len(sub_df))

            return self.UMAP

    sss = UMAPPlotClass(name=titulo)
    panel_row = panel.Row(panel.Param(sss.param, widgets={'obj_opciones': {'name': 'Opciones'},
                                                          'obj_autonomias': {'size': 19, 'name': 'Autonomías'}
                                                          }), sss.update_df_datos)
    return panel_row

In [ ]:
df_datos_secciones = crear_df_datos(municipios_gpd, 0.1, 0.2, seed=10, r_min=2, r_max=15, )

In [ ]:
dibujar_UMAP_votos_autonomia(df_datos_secciones, dict_colores, alpha_max=0.95, alpha_min = 0.005, titulo='')

### Análisis de municipios
El análisis de municipios da similares al de las secciones. Las variaciones más significativas son que los clusters de Canarias y Melilla se han absorbido al cluster principal, aunque sí que es cierto que el de canarias sale como una rama, luego podríamos decir que se trata de una separación parcial.

Para los clusters separados las distinciones del UMAP son similares. Por ejemplo, en Euskadi seguimos viendo la tendencia de municipios con mayoria EH Bildu, que suelen ser poblaciones más pequeñas, y en el otro extremo los municipios con mayoria PSOE, que suelen ser ciudades más grandes.

Para Cataluña vemos unos resultados similares: Las grandes capitales y ciudades se encuentran en el cluster 11, con mayoría PSOE, o empate entre PSOE y ERC. En las otras dos puntas del triángulo tenemos los pueblos con mayoría de ERC o de JxCAT. En el centro tenemos pueblos y ciudades pequeñas con mezcla de los tres partidos.

Si miramos el gran cluster central vemos los clusters de Canarias y la Comunidad Valenciana, con la tónica general de que cuanto más alejado está del cluster central, más presencia de los partidos regionalistas hay. Ahora queda bastante más claro la existencia de las dos Españas (por `partido_1` para verlo más fácil) 

Una cosa que resulta curiosa es que los dos extremos se encuentren en un mismo cluster: en el cluster 4 aparecen tanto los municipios con mayoría de UP, como los que tienen mayoría de VOX; aunque ambos extremos sean frontera con otros clusters. Como norma general, cuanto más a las afueras del cluster, más radical el voto. Por ejemplo, en las puntas de mayoría de PP hay porcentajes de entre rl 60% y el 100% [Valdeprado, CyL, 8 habitantes]. En estas zonas del cluster se encuentran, mayoritariamente, Galicia (pueblos grandes) y Castilla y León (pueblos más pequeños). En el otro extremo también se ven mayorías del PSOE con valores entre el 60 y el 80%. La línea fronteriza tiene puntos mixtos donde PSOE y PP tienen porcentaje similares. Por último, al otro extremo hay puntos con elementos de Cs y Podemos. Así, con esos 3 puntos clave podemos ir haciendo mezclas. Por ejemplo, en la punta de Cs se ven las mezclas PSOE + Cs + VOX, y en la parte verde y todo el extremo del cluster se ve la influencia del trifachito. En la línea fronteriza entre PP y PSOE, cerca de VOX, se observa la mezcla PP + PSOE + VOX. Estas dos opciones son las que marcan la frontera entre el azul y el rojo, y se extienden incluso hasta los centros de ambos partidos.

Si se pone el mapa en `Autonomia` está guay ver como se marca la división entre PP y PSOE con las autonomias. Se ve clarísimo que PP está gobernado por CyL y ClM, y la mitad PPera de Galicia, mientras que los municipios de PSOE pertenecen a Andalucía, Extremadura y una parte de Galicia. Los pueblos con mayoria de Ciudadanos están localizados en Madrid.

En cualquiera de los dos casos (municipios o secciones) tenemos que Canarias y la Comunidad Valenciana están más cerca del cluster central que el resto de autonomías. Con esta técnica de reducción de dimensiones podemos decir que, como están más cerca que el resto, la fuerza de los partidos regionalistas es bastante menor que en el resto de autonomías. Esto puede ser bien porque la presencia de estos partidos es minoritaria y no obtienen porcentajes muy altos (máximo 30% de votos VS > 50 o 60% para partidos como EH Bildu o ERC en ciertos municipios/secciones), o porque estas autonomías presentan más de un partido regionalista. Por ejemplo, Cataluña tiene a ERC y JxCAT, Euskadi tiene a EH Bildu y PNV-EAJ, Navarra tiene a NA+ y EH Bildu. En este caso, dos partidos son dos dimensiones, luego es más fácil que se separen del cluster central que una sección / municipio que se distinga por un partido.

**Para la web hacer dibujos explicativos con flechas y áreas de las principales inluencias**

In [ ]:
df_datos_municipios = eec.crear_df_datos(municipios_gpd, 0.1, 0.2, seed=10, r_min=2, r_max=15, )
eec.dibujar_UMAP_votos_autonomia(df_datos_municipios, dict_colores, alpha_max=0.95, alpha_min = 0.025, titulo='')

In [ ]:
from IPython.display import IFrame

IFrame(src='https://jsfiddle.net/alexmascension/xnkb14hw/135/embedded/result/', width=970, height=700)

In [ ]:
autonomias = sorted(list(dict.fromkeys(dict_prov.values())))

## ¡Hora de correlaciones!
Para ver algunas tendencias en los votos vamos a realizar varias correlaciones sencillas. Las dos variables más directas que se pueden correlacionar con los datos de las votaciones son 1) La población y 2) El porcentaje de abstención. Para la población sólo tomaremos los municipios, porque las secciones "capan" la población y se pierde representatividad.

#### Preparado de datos

In [ ]:
df_scatter_autonomia = municipios_gpd.copy()
list_porcentajes, list_partidos, list_colores = [], [], []
cols_porcentajes = [i for i in df_scatter_autonomia.columns if (i.startswith('porcentaje_')) & 
                    ((not i[-1].isdigit()))]

for partido in cols_porcentajes:
    nombre_partido = partido.split('_')[-1]
    list_porcentajes += df_scatter_autonomia[partido].values.tolist()
    list_partidos += [nombre_partido] * len(df_scatter_autonomia)
    list_colores += [dict_colores[nombre_partido] if nombre_partido in dict_colores else "#bbbbbb"] * len(df_scatter_autonomia)

df_source = pd.DataFrame({'partido': list_partidos, 'porcentaje':list_porcentajes, 'color':list_colores, 
                         'municipio': df_scatter_autonomia['Nombre'].values.tolist() * len(cols_porcentajes) , 
                         'poblacion': df_scatter_autonomia['censo escrutinio'].values.tolist() * len(cols_porcentajes),
                         'logpob': np.log10(df_scatter_autonomia['censo escrutinio'].values).tolist() * len(cols_porcentajes),
                         'provincia': df_scatter_autonomia['CPRO'].values.tolist() * len(cols_porcentajes), 
                         'autonomia': df_scatter_autonomia['Autonomia'].values.tolist() * len(cols_porcentajes), 
                         'alpha': [0.7] * len(df_scatter_autonomia) * len(cols_porcentajes),
                         'porcentaje_abstencion': df_scatter_autonomia['porcentaje_abstencion'].values.tolist() * len(cols_porcentajes)})
df_source = df_source[df_source['porcentaje'] > 0]


### Correlación con tamaño de población
El heatmap muestra algunas correlaciones interesantes. Generalmente Cs se ha favorecido de grandes poblaciones el que más, mostrando altas correlaciones en Asturias, Baleares, Galicia y Murcia. Unidas Podemos ha tenido un mayor porcentaje de votos en Galicia. El PSOE también ha salido votado en las poblaciones más grandes de Euskadi y Cataluña, aunque esto ya es evidente de por sí por una mayor proporción de población extranjera en estas ciudades. Los porcentajes de abstención no están relacionados con el tamaño poblacional, quitando en Baleares, que tiene una correlación positiva.

Si nos vamos al lado opuesto, EH Bildu, ERC y JxCAT son los más perjudicados por el tamaño poblacional, aunque no es extraño por la misma explicación que antes. Si nos centramos en autonomías sin partidos regionalistas, el PP ha sido el partido más afectado en los municipios más grandes, sobre todo en Galicia y Murcia.

**Nota: Las correlaciones están exageradas por tomar las poblaciones en logaritmo. Si en lugar de `logpob` se pone `poblacion` se obtienen las correlaciones verdaderas, que son casi la mitad, y mucho más ruidosas.** Aun así, las correlaciones siguen siendo estadísticamente significativas, pero hay que notar que la magnitud se reduce.

De hecho, si analizamos las correlaciones con los valores poblacionales reales, el ajuste lineal no es el ideal. Sin embargo, sí que la mayoría de las conclusiones iniciales se mantiene bien.

In [ ]:
eec.scatter_multiselect_subordinada(df_source, 'autonomia', 'partido', 'logpob', 'porcentaje', corr_type='exp')

In [ ]:
show(eec.heatmap_correlaciones(df_source, 'partido', 'autonomia', 'logpob', 'porcentaje', cmap='seq'))

In [ ]:
show(eec.heatmap_correlaciones(df_source, 'partido', 'autonomia', 'poblacion', 'porcentaje', corr_type='exp', 
                              cmap='seq'))

### Correlación entre el porcentaje de abstención y voto
No hay correlaciones evidentes en la gran mayoría de casos. Sin embargo, sí que es cierto que el porcentaje de votos al PP disminuye considerablemente en Murcia, es decir, municipios con menos abstención han votado más al PP en Murcia, con Aledo, Ulea y Ojós los más importantes. ¿Qué repercusiones tiene esto en las elecciones? Poco. Pero al PP le puede convenir hacer más campaña en Murcia sabiendo que las abstenciones no le favorecen. 

En otros casos sucede lo contrario. Por ejemplo, en Canarias el porcentaje de abstención favorece el porcentaje de votos a VOX, incluso en municipios grandes como Arona o Adeje. 

Para el resto de casos las abstenciones no tienen mucha relevancia en predecir el porcentaje de votos. Por ejemplo, en Euskadi, PNV se lleva menos votos en pueblos con mayor abstención; pero el tamaño de esos pueblos es pequeño en comparación con las capitales y pueblos grandes, donde hay más votos. Así que la correlación no es relevante.

In [ ]:
show(eec.heatmap_correlaciones(df_source[df_source['partido'] != 'abstencion'], 'partido', 'autonomia', 
                               'porcentaje', 'porcentaje_abstencion', cmap='seq'))

In [ ]:
eec.scatter_multiselect_subordinada(df_source[df_source['partido'] != 'abstencion'], 
                                    'autonomia', 'partido', 'porcentaje_abstencion', 'porcentaje',  
                                   )

## Correlaciones demográficas
Ahora vamos a tomar varias variables demográficas y vamos a incluirlas en el análisis. Principalmente vamos a buscar correlaciones entre edad, renta, porcentaje de hombres, y resto de variables disponibles para buscar alguna correlación en los resultados de las elecciones. Casi seguramente que las correlaciones en bruto no salgan y habrá que hacer selecciones para determinar alguna tendencia.


Por facilitar el análisis, primero cargaremos los datos demográficos y de renta más importantes en el dataframe.

## Guardado de datos
Vamos a guardar los datos para no tener que cargarlos de nuevo, o para que puedan ser usados en otros notebooks del mismo nivel o nivel superior.

In [ ]:
municipios_gpd = gpd.read_file(current_dir + '/datos/mapas_elecciones/porcentaje_mas_votado_2019_04_municipios.json', driver='GeoJSON', encoding='utf-8')
secciones_gpd = gpd.read_file(current_dir + '/datos/mapas_elecciones/porcentaje_mas_votado_2019_04_secciones.json', driver='GeoJSON', encoding='utf-8')
provincias_gpd = gpd.read_file(current_dir + '/datos/mapas_elecciones/porcentaje_mas_votado_2019_04_provincias.json', driver='GeoJSON', encoding='utf-8')